# E-Commerce Clothing Classifer
## Image classification using convolutional neural networks in PyTorch

### Import modules

In [2]:
import torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Using data from FashionMNIST publically available clothing image datasets

In [3]:
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform = transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [4]:
classes = train_data.classes
num_classes = len(train_data.classes)
print(f"classes: {classes}\nnum_classes: {num_classes}")

classes: ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
num_classes: 10


In [5]:
num_input_channels = 1
num_output_channels = 16
image_size = train_data[0][0].shape[1]
image_size

28

In [14]:
class MultiClassImageClassifier(nn.Module):
  
    def __init__(self, num_classes):
        super(MultiClassImageClassifier, self).__init__()

        self.classifier = nn.Sequential(
            nn.Conv2d(num_input_channels, num_output_channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(num_output_channels * (image_size//2)**2, num_classes) # Create a fully connected layer
        )
        
    def forward(self, x):
        # Pass inputs through each layer
        x = self.classifier(x)
        return x

In [15]:
dataloader_train = DataLoader(train_data, shuffle=True, batch_size = 10,)

In [16]:
def train_model(optimizer, net, num_epochs):
    num_processed = 0
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        running_loss = 0.0
        num_processed = 0
        for feature,label in dataloader_train:
            optimizer.zero_grad()
            outputs = net(feature)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            num_processed += len(label)
        print(f"epoch:{epoch}, loss:{running_loss/num_processed}")
        
    train_loss = running_loss / len(dataloader_train)

In [17]:
net = MultiClassImageClassifier(num_classes)
optimizer = optim.Adam(net.parameters(), lr = 0.001)

In [18]:
train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=1,
)

epoch:0, loss:0.04287870064123999


In [19]:
dataloader_test = DataLoader(test_data,
                            shuffle = False,
                            batch_size =10,)

In [20]:
# Define the metrics
accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes)
precision_metric = Precision(task='multiclass', num_classes=num_classes, average=None)
recall_metric = Recall(task='multiclass', num_classes=num_classes, average=None)

# Run model on test set
net.eval()
predicted = []
for i, (features, labels) in enumerate(dataloader_test):
    output = net.forward(features.reshape(-1, 1, image_size, image_size))
    cat = torch.argmax(output, dim=-1)
    predicted.extend(cat.tolist())
    accuracy_metric(cat, labels)
    precision_metric(cat, labels)
    recall_metric(cat, labels)

# Compute the metrics
accuracy = accuracy_metric.compute().item()
precision = precision_metric.compute().tolist()
recall = recall_metric.compute().tolist()
print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)

Accuracy: 0.8744999766349792
Precision (per class): [0.7773882746696472, 0.9906736016273499, 0.846069872379303, 0.842056930065155, 0.7206119298934937, 0.981249988079071, 0.7776097059249878, 0.9347389340400696, 0.9805725812911987, 0.9211026430130005]
Recall (per class): [0.8870000243186951, 0.9559999704360962, 0.7749999761581421, 0.9169999957084656, 0.8949999809265137, 0.9419999718666077, 0.5139999985694885, 0.9309999942779541, 0.9589999914169312, 0.968999981880188]
